In [3]:
# import wave
# import torch
import time
import threading
from TTS.api import TTS
# import requests
import sounddevice as sd
# import json
# from IPython.display import Audio
# from playsound import playsound
# import pygame
# import numpy as np
# import os
import queue
# from pydub import AudioSegment
# import os

In [4]:
#Coqui TTS
# tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False, gpu=False)

tts = TTS(model_name="tts_models/en/ljspeech/vits", progress_bar=False, gpu=False)

 > Downloading model to /Users/anshulwadhawan/Library/Application Support/tts/tts_models--en--ljspeech--vits


In [3]:
#Eleven Labs
url = "https://api.elevenlabs.io/v1/text-to-speech/TxGEqnHWrfWFTfGW9XjX"
headers = {
    "xi-api-key": "ff79eb4ad43bb12b04a3968a5b92125b"
}

In [3]:
text ="A muffin is an individually portioned baked product, however the term can refer to one of two distinct items: a part-raised flatbread that is baked and then cooked on a griddle, or a quickbread that is chemically leavened and then baked in a mold."
text_small = "Muffin is a baked item, either a flatbread cooked on griddle or a quickbread leavened chemically and baked in a mold."
text_hi = "Hi"

In [8]:
q = queue.Queue()
done = False
# pygame.mixer.init()

def streaming_write(sentence):
    for word in sentence.split():
        with open("streaming_input.txt", "a") as file:
            file.write(word+" ")
        time.sleep(0.2)

def convert_to_audio(chunk, name):
    # samples = tts.synthesize(chunk, speaker)
    # with wave.open("outputs/example"+ str(name) +".wav", "w") as fp:
    #     fp.setparams((1, 2, 24000, len(samples), "NONE", "NONE"))
    #     fp.writeframes(samples)

    filepath = "outputs/example"+ str(name) +".wav"
    start = time.time()
    tts.tts_to_file(chunk, speaker=tts.speakers[0], language="en", file_path=filepath)
    # data = {
    #     "text": chunk,
    #     "voice_settings": {
    #         "stability": 0,
    #         "similarity_boost": 0
    #     }
    # }

    # response = requests.post(url, data=json.dumps(data), headers=headers)

    # if response.status_code == 200:
    #     with open(filepath, 'wb') as f:
    #         f.write(response.content)
    # else:
    #     print("Error")

    # pygame.mixer.music.load(filepath)
    # pygame.mixer.music.play()
    # while pygame.mixer.music.get_busy():
    #     # pygame.time.Clock().tick(10)
    #     continue

    end = time.time()
    print(end-start)
    q.put(filepath)
    
def streamingTTS():
    content=""
    while True:
        with open("streaming_input.txt", "r") as file:
            new_content = file.read()
            print(len(content), len(new_content))
            if len(new_content) > len(content):
                convert_to_audio(new_content[len(content):], len(new_content))
                content = new_content
            else:
                done = True
                break
        # time.sleep(0.3)

def audioplayer():
    pygame.mixer.init()
    while not done:
        while not q.empty():
            filepath = q.get()
            pygame.mixer.music.load(filepath)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                continue


In [9]:
write_thread = threading.Thread(target=streaming_write, args = (text,) )
write_thread.start()

# write_thread = threading.Thread(target=streaming_write, args = (text_small,) )
# write_thread.start()

In [10]:
player_thread = threading.Thread(target=audioplayer)
player_thread.start()

In [11]:
tts_thread = threading.Thread(target=streamingTTS)
tts_thread.start()

0 12
 > Text splitted to sentences.
['A muffin is']


 > Processing time: 6.867577791213989
 > Real-time factor: 4.28955514754153
6.932184219360352
12 181
 > Text splitted to sentences.
['an individually portioned baked product, however the term can refer to one of two distinct items: a part-raised flatbread that is baked and then cooked on a griddle, or']
 > Processing time: 2.310619831085205
 > Real-time factor: 0.2136100426259781
2.4083380699157715
181 248
 > Text splitted to sentences.
['a quickbread that is chemically leavened and then baked in a mold.']
 > Processing time: 0.9408981800079346
 > Real-time factor: 0.21855939140718575
0.9485330581665039
248 248
